## Linear model

using a linear model to predict the energy consumption

In [29]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

from data_read import load_data_into_dataframe
from house_filtration import house_filtration_select

import datetime
import time

In [27]:
# Load the dataset
df = house_filtration_select()


d:\OneDrive - Universitetet i Stavanger\Bachelor\data_read.py:49: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(file) for file in dataset_files_csv), ignore_index=False)
d:\OneDrive - Universitetet i Stavanger\Bachelor\data_read.py:49: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(file) for file in dataset_files_csv), ignore_index=False)
d:\OneDrive - Universitetet i Stavanger\Bachelor\data_read.py:49: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(file) for file in dataset_files_csv), ignore_index=False)
d:\OneDrive - Universitetet i Stavanger\Bachelor\data_read.py:49: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(file) for file in dataset_files_csv), igno

       LCLid                         tstp energy(kWh/hh)
0  MAC000002  2012-10-12 00:30:00.0000000             0 
1  MAC000002  2012-10-12 01:00:00.0000000             0 
2  MAC000002  2012-10-12 01:30:00.0000000             0 
3  MAC000002  2012-10-12 02:00:00.0000000             0 
4  MAC000002  2012-10-12 02:30:00.0000000             0 
-80.21616768836975


NameError: name 'datetime' is not defined

In [30]:
def unix_time(timestamp):
    values = timestamp.split(" ")
    date = values[0].split("-")
    year = int(date[0])
    month = int(date[1])
    day = int(date[2])
    date = datetime.date(year, month, day)
    clock = values[1].split(":")
    hour = int(clock[0])
    minute = int(clock[1])
    second = int(0)
    clock = datetime.time(hour, minute, second)
    return float(datetime.datetime.combine(date, clock).timestamp())

df["tstp"] = df["tstp"].apply(unix_time)

In [66]:
df.head()

,LCLid,tstp,energy(kWh/hh)
233272,MAC000165,1.322132e+09,0.114
233273,MAC000165,1.322134e+09,0.174
233274,MAC000165,1.322136e+09,0.112
233275,MAC000165,1.322138e+09,0.125
233276,MAC000165,1.322140e+09,0.597


In [67]:
df["energy(kWh/hh)"] = pd.to_numeric(df["energy(kWh/hh)"], downcast="float", errors="coerce")


# id for the household we want to use to train and test
id_training = 'MAC000150' 
id_eval = 'MAC000152'


def find_household(identification):
    affluenthh_filt = df["LCLid"] == identification
    affluent_hh_data = df[affluenthh_filt]
    return affluent_hh_data

In [40]:
training = find_household(id_training)
eval = find_household(id_eval)

dftrain = training
dfeval = eval

y_train = dftrain.pop("energy(kWh/hh)")
y_eval = dfeval.pop("energy(kWh/hh)")

In [60]:
dftrain.head()

,LCLid,tstp
69303,MAC000150,1.322039e+09
69304,MAC000150,1.322041e+09
69305,MAC000150,1.322042e+09
69306,MAC000150,1.322044e+09
69307,MAC000150,1.322046e+09


In [69]:
dftrain.shape

(39720, 2)

In [43]:
dftrain.describe(datetime_is_numeric=True)

,tstp
count,3.972000e+04
mean,1.357789e+09
std,2.064149e+07
min,1.322039e+09
25%,1.339912e+09
50%,1.357790e+09
75%,1.375663e+09
max,1.393542e+09


In [44]:
y_train.head()

69303    0.301
69304    0.127
69305    0.201
69306    0.564
69307    0.159
Name: energy(kWh/hh), dtype: float32

In [72]:
CATEGORIAL_COLUMNS = ['LCLid']
NUMERIC_COLUMNS = ['tstp']

feature_columns = []
for feature_name in CATEGORIAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='LCLid', vocabulary_list=('MAC000150',), dtype=tf.string, default_value=-1, num_oov_buckets=0), NumericColumn(key='tstp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [73]:
#check https://www.tensorflow.org/tutorials/estimator/linear

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use


train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)


In [74]:
#Creating the model

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\denny\\AppData\\Local\\Temp\\tmpw52ttwqo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [75]:
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


D:\Python\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\denny\AppData\Local\Temp\tmpw52ttwqo\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0


InvalidArgumentError: Graph execution error:

assertion failed: [Labels must be <= n_classes - 1] [Condition x <= y did not hold element-wise:] [x (head/losses/labels:0) = ] [[0.166][0.231][0.228]...] [y (head/losses/check_label_range/Const:0) = ] [1]
	 [[{{node Assert}}]]